# "Concevez une application au service de la santé publique"
_Cleaning Notebook_
13/08 16h

## 0 Preliminaries

### 0.0 Importing Modules

In [0]:
# # Importing requirements.txt file
# from google.colab import files
# files.upload()

In [2]:
import sys
# Checking whether the notebook is on Colab or PC
is_colab = 'google.colab' in sys.modules
is_colab, sys.executable

(True, '/usr/bin/python3')

In [3]:
# Replaced by 
# !pip install -r requirements.txt
# Installations and importations required for Google Colab version
if is_colab==True:
  from google.colab import files, output
  !pip install python-stdnum
  !pip install gTTS
  !pip install ipywidgets
else:
    pass

     |████████████████████████████████| 849kB 2.8MB/s 
  Created wheel for gtts-token: filename=gTTS_token-1.1.3-cp36-none-any.whl size=4097 sha256=c4ead1ba992431268915fb7a5aa0bd783ddc844819eaa2736d1776f36e7129b3
  Stored in directory: /root/.cache/pip/wheels/dd/11/61/33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34
Successfully built gtts-token


In [4]:
## IMPORTATIONS
import os
import io
import json
import re
import pandas as pd
import missingno as msno
from stdnum.exceptions import *
from stdnum.util import *
from stdnum.ean import *
import numpy as np
from ipywidgets import interact, interact_manual

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Function to display small dataframes side by side
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),
                 raw=True)

In [0]:
# Setting pandas display options 
dict_pd_settings = {'display.max_rows': 500, 'display.width': 100,
                    'display.max_colwidth': 20,
                    'display.float_format': lambda x: '%.0f' % x}
for k,v in dict_pd_settings.items():
  pd.set_option(k,v)

In [0]:
# To play audio text-to-speech during execution
from IPython.display import Audio
from gtts import gTTS

def speak(my_text):
    with io.BytesIO() as f:
        gTTS(text=my_text, lang='en').write_to_fp(f)
        f.seek(0)
        return Audio(f.read(), autoplay=True)

In [8]:
speak('Packages and modules successfully imported')

### 0.1 Importing a sample of the dataset

In order to facilitate further manipulation of the dataset, we only upload a sample of the dataset at fisrt.

This will allow us to determine the best encoding form for each column therefore resulting in a better optimization of memory usage while uploading the full dataset.

In [9]:
p = 0.005  # reads 0.5% of the lines

if is_colab==False:
  # Importing database from my current local Git Repo
  df = pd.read_csv('../DATA/en.openfoodfacts.org.products.csv',
         header=0, sep='\t', low_memory=False, encoding ='utf-8',
         skiprows=lambda i: i>0 and np.random.random() > p)
  print("Imported from PC")
else:
  # Importing database from the Internet
  if not os.path.exists('en.openfoodfacts.org.products.csv'):
    !wget https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv
  else:
    pass
  df = pd.read_csv('/content/en.openfoodfacts.org.products.csv',
         header=0, sep='\t', low_memory=False, encoding ='utf-8',
         skiprows=lambda i: i>0 and np.random.random() > p)
  print("Imported from colab environment")

--2020-04-23 17:25:38--  https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv
Resolving static.openfoodfacts.org (static.openfoodfacts.org)... 213.36.253.206
Connecting to static.openfoodfacts.org (static.openfoodfacts.org)|213.36.253.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2549543567 (2.4G) [text/csv]
Saving to: ‘en.openfoodfacts.org.products.csv’

en.openfoodfacts.or 100%[===================>]   2.37G  11.6MB/s    in 3m 39s  

2020-04-23 17:29:19 (11.1 MB/s) - ‘en.openfoodfacts.org.products.csv’ saved [2549543567/2549543567]

Imported from colab environment


In [10]:
speak('Dataset successfully imported')

### 0.2 Memory usage reduction

In order to facilitate the manipulation, we change the format of encoding
- ints and floats are to be downcasted
- objects are to be changed to category (if the number of unique values is less tha, half the number of rows)

NB: If we already know the format of data, we can specify the format of each column while loading the data in a pandas dataframe: *pd.read_csv('path', dtype=my_dict)*

In [11]:
''' This cell is a demo showing the performance of a good choice of data type
while reading the csv to dataframe - not necessary to run each time
'''
# def mem_usage(pandas_obj):
#   if isinstance(pandas_obj,pd.DataFrame):
#     usage_b = pandas_obj.memory_usage(deep=True).sum()
#   else:
#     usage_b = pandas_obj.memory_usage(deep=True)    
#   usage_mb = usage_b / 1024 ** 2
#   return "{:03.2f} MB".format(usage_mb)

# # Checking memory usage for the sample
# print(df.info(memory_usage='deep'), '\n'+'.'*20)

# # columns 'category'
# df_obj = df.select_dtypes(include=['object']).copy()
# converted_obj = pd.DataFrame()
# for col in df_obj.columns:
#   if len(df_obj[col].unique()) / len(df_obj[col]) < 0.5:
#     converted_obj.loc[:,col] = df_obj[col].astype('category')
#   else:
#     converted_obj.loc[:,col] = df_obj[col]
# print('For {} object columns: {} before, {} after conversion to category.'\
#       .format(df_obj.shape[1], mem_usage(df_obj), mem_usage(converted_obj)))
# # columns 'int'
# df_int = df.select_dtypes(include=['int']).copy()
# converted_int = df_int.apply(pd.to_numeric, downcast='unsigned')
# print('For {} int columns: {} before, {} after downcasting.'\
#       .format(df_int.shape[1], mem_usage(df_int), mem_usage(converted_int)))
# # columns 'float'
# df_float = df.select_dtypes(include=['float']).copy()
# converted_float = df_float.apply(pd.to_numeric, downcast='float')
# print('For {} float columns: {} before, {} after downcasting.'\
#       .format(df_float.shape[1], mem_usage(df_float), mem_usage(converted_float),))

# df[converted_obj.columns] = converted_obj
# df[converted_int.columns] = converted_int
# df[converted_float.columns] = converted_float
# print('.'*20+'\n')

# # Re-checking memory usage for the sample 
# print('.'*20+'\n', df.info(memory_usage='deep'))
# # (60.9MB->26.0MB: less than 43% !!!)

# del df_obj, converted_obj, df_int, converted_int, df_float, converted_float

' This cell is a demo showing the performance of a good choice of data type\nwhile reading the csv to dataframe - not necessary to run each time\n'

In [0]:
# display(df.head(3), df.dtypes.value_counts())
# print(df.dtypes)

In [13]:
## Changing format of object and float64 columns routine

my_dtypes = {}

# object -> category (if nb unique < half total nb of rows)
for col in df.columns.values:
    if df[col].dtype == 'object' :
      if len(df[col].unique()) / len(df[col]) < 0.5:
        my_dtypes[col] = 'category'
      else:
        my_dtypes[col] = 'object'

# float64 -> float32
for col in df.columns.values:
    if df[col].dtype == 'float64':
      if df[col].notna().sum()!=0: # 1% sample columns happening to be all null 
        my_dtypes[col] = 'float32'
      else:
        my_dtypes[col] = 'category'

# float64 -> float32
for col in df.columns.values:
    if df[col].dtype == 'int64':
        my_dtypes[col] = 'int8'
        
print(my_dtypes)

{'code': 'object', 'url': 'object', 'creator': 'category', 'created_datetime': 'object', 'last_modified_datetime': 'object', 'product_name': 'object', 'generic_name': 'category', 'quantity': 'category', 'packaging': 'category', 'packaging_tags': 'category', 'brands': 'category', 'brands_tags': 'category', 'categories': 'category', 'categories_tags': 'category', 'categories_en': 'category', 'origins': 'category', 'origins_tags': 'category', 'manufacturing_places': 'category', 'manufacturing_places_tags': 'category', 'labels': 'category', 'labels_tags': 'category', 'labels_en': 'category', 'emb_codes': 'category', 'emb_codes_tags': 'category', 'first_packaging_code_geo': 'category', 'cities_tags': 'category', 'purchase_places': 'category', 'stores': 'category', 'countries': 'category', 'countries_tags': 'category', 'countries_en': 'category', 'ingredients_text': 'category', 'allergens': 'category', 'traces': 'category', 'traces_tags': 'category', 'traces_en': 'category', 'serving_size': 

In [14]:
# Importing the whole dataframe with optimized types

if is_colab==False:
  # Importing database from my current local Git Repo
  df = pd.read_csv('../DATA/en.openfoodfacts.org.products.csv', dtype=my_dtypes,
         header=0, sep='\t', low_memory=False, encoding ='utf-8')
  print("Imported from PC")
else:
  df = pd.read_csv('/content/en.openfoodfacts.org.products.csv', dtype=my_dtypes,
         header=0, sep='\t', low_memory=False, encoding ='utf-8')
  print("Imported from colab environment")

Imported from colab environment


In [15]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1219248 entries, 0 to 1219247
Columns: 181 entries, code to carnitine_100g
dtypes: category(96), float32(76), int8(2), object(7)
memory usage: 2.0 GB


In [16]:
speak("Reduction of memory usage done")

Main conclusions : 
- Description :
  954463 rows and 175 columns
  some of them categorical, 
  some of them numerical
- Memory usage reduction : 
  6.0 GB -> 2.2 GB

### 0.3 First Overview

In [17]:
# Shape of the dataset
dfshape = df.shape # (1173417, 178) (954463, 175)
dfshape

(1219248, 181)

In [18]:
# General description 
desc_df = df.describe(include='all')

# Types and content of the columns
desc_type = pd.DataFrame(df.dtypes, columns=['type']).T
nb_null = (dfshape[0]-desc_df.loc["count"].T)
pct_null = (dfshape[0]-desc_df.loc["count"].T)*100/dfshape[0]
desc_null = pd.DataFrame([nb_null, pct_null], index=['nb null','pct null'])

# Concatenate general, type and null stats
my_desc = pd.concat([desc_type, desc_null, desc_df])
my_desc

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,...,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
type,object,object,category,int8,object,int8,object,object,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,...,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,category,category,float32,float32,float32,float32,float32,float32,float32,float32,float32,category,float32,float32,float32,float32,category,category,category,float32,category,category,category
nb null,0,0,4,0,0,0,0,59827,1119788,839176,1006846,1006855,459556,459605,673207,673210,673210,1160234,1160333,1125281,1125318,912581,912560,912560,1144156,1144184,1171536,1219248,1167979,1086936,1022687,2122,2124,2124,736401,1115439,1219248,1146178,1125162,1125162,...,1214071,1211367,1218574,1215795,1219144,1218921,1184059,1218653,1068104,1210857,1072064,1209747,1213186,1216832,1217176,1218975,1217842,1219175,1219166,1218286,1218956,1219126,1219098,1213235,1218960,1208183,1218957,1214084,1219246,1218835,1209194,795806,1219204,1219245,1219248,1219211,1218744,1219226,1219209,1219231
pct null,0,0,0,0,0,0,0,5,92,69,83,83,38,38,55,55,55,95,95,92,92,75,75,75,94,94,96,100,96,89,84,0,0,0,60,91,100,94,92,92,...,100,99,100,100,100,100,97,100,88,99,88,99,100,100,100,100,100,100,100,100,100,100,100,100,100,99,100,100,100,100,99,65,100,100,100,100,100,100,100,100
count,1219248,1219248,1219244,1219248,1219248,1219248,1219248,1159421,99460,380072,212402,212393,759692,759643,546041,546038,546038,59014,58915,93967,93930,306667,306688,306688,75092,75064,47712,0,51269,132312,196561,1217126,1217124,1217124,482847,103809,0,73070,94086,94086,...,5177,7881,674,3453,104,327,35189,595,151144,8391,147184,9501,6062,2416,2072,273,1406,73,82,962,292,122,150,6013,288,11065,291,5164,2,413,10054,423442,44,3,0,37,504,22,39,17
unique,1219011,1219086,10859,NaN,1072250,NaN,940993,794325,73399,30960,37072,30019,131752,104362,87614,53008,53008,10198,9006,21331,19303,57252,42731,42731,21967,19381,2953,0,5049,9742,11654,7718,2842,2842,424599,5639,0,8810,8896,8896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,3,0,24,NaN,14,25,10
top,8718885890051,http://world-en....,kiliweb,NaN,2017-03-09T16:32...,NaN,2019-03-16T21:14...,Aceite de oliva ...,Pâtes alimentair...,500 g,"sachet,plastique","sachet,plastique",Carrefour,carrefour,Snacks,en:snacks,Snacks,France,france,France,france,en:gluten-free,en:gluten-free,Gluten-free,FR 72.264.002 EC,fr-72-264-002-ec,"47.833333,-0.333333",NaN,sable-sur-sarthe...,France,Carrefour,France,en:france,Fra

In [19]:
print(tuple(df.columns.values))

('code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutriments', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_tags', 'nutriscore_score', 'nutriscore_grade', '

14 columns with no entry, no row has all the columns filled.

In [20]:
speak('Description overview done')

### 0.4 Pandas Data Profiling report

In [0]:
# Generating a Data Profiling html report via pandas_profiling
# profile = pandas_profiling.ProfileReport(df) #, check_correlation = False
# profile.to_file("OFFacts_report2_3.html")
# profile

In [0]:
# # extract rejected columns (based on correlation coeff greater than 0.9)
# rejected_variables_90 = profile.get_rejected_variables(threshold=0.9)
# len(rejected_variables_90)

In [0]:
# speak('Pandas profiling report generated')

Main conclusions : 

## 1 Dropping columns and rows

### 1.1 Filtering out non relevant columns
Some columns can immediately be classified as non useful for our project and therefore dropped : 
- 5 columns concerning to the image of the product: 'image_small_url',	'image_ingredients_url',	'image_ingredients_small_url',	'image_nutrition_url'	'image_nutrition_small_url'
- 3 columns concerning the level of completion of the row: 'states', 'states_tags', 'states_en' 


In [24]:
li_col_del = ['image_url', 'image_small_url',	'image_ingredients_url',
              'image_ingredients_small_url',	'image_nutrition_url',
              'image_nutrition_small_url',
              'states', 'states_tags', 'states_en' ]
# Filtering out 8 non relevant columns
df.drop(columns=li_col_del, inplace=True)
df.shape

(1219248, 172)

### 1.2 Almost empty columns

Let's see how the columns are filled.


In [25]:
# Checking nb of not null values in the columns
nbnotnulldf = pd.DataFrame((~df.isna()).sum(axis =0), columns=['nb'])
nbnotnulldf.sort_values(by=['nb'], axis=0, ascending=True, inplace=True)
nbnotnulldf.T.head(150)

,ingredients_from_palm_oil,cities,ingredients_that_may_be_from_palm_oil,-lignoceric-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,allergens_en,-melissic-acid_100g,no_nutriments,-caproic-acid_100g,water-hardness_100g,-butyric-acid_100g,-dihomo-gamma-linolenic-acid_100g,-caprylic-acid_100g,-myristic-acid_100g,-stearic-acid_100g,chlorophyl_100g,-montanic-acid_100g,glycemic-index_100g,-cerotic-acid_100g,-palmitic-acid_100g,-capric-acid_100g,-gamma-linolenic-acid_100g,-behenic-acid_100g,-lauric-acid_100g,-maltose_100g,nucleotides_100g,carnitine_100g,-maltodextrins_100g,-arachidic-acid_100g,-oleic-acid_100g,beta-glucan_100g,-arachidonic-acid_100g,additives,serum-proteins_100g,choline_100g,inositol_100g,...,labels_tags,fiber_100g,quantity,nova_group,nutriscore_score,nutriscore_grade,nutrition-score-fr_100g,ingredients_text,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,main_category,categories_en,categories_tags,main_category_en,categories,brands_tags,brands,energy-kcal_100g,saturated-fat_100g,sugars_100g,sodium_100g,salt_100g,carbohydrates_100g,fat_100g,proteins_100g,energy_100g,product_name,pnns_groups_1,countries_en,countries_tags,countries,pnns_groups_2,creator,last_modified_t,created_datetime,created_t,url,last_modified_datetime,code
nb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2,3,3,3,4,5,7,9,9,13,17,17,21,22,22,29,29,36,37,39,...,306688,318855,380072,391266,423433,423433,423442,482847,482847,482847,482847,546038,546038,546038,546038,546041,759643,759692,889132,922913,939902,940374,940395,954246,954270,955750,961845,1159421,1206218,1217124,1217124,1217126,1218618,1219244,1219248,1219248,1219248,1219248,1219248,1219248


In [26]:
# Displaying a table with bins
mask = pd.cut(nbnotnulldf['nb'], [-1,0,5,10,100,300,1000000]) # 
mask.value_counts(normalize=False, sort=False)

(-1, 0]            14
(0, 5]             12
(5, 10]             3
(10, 100]          19
(100, 300]         11
(300, 1000000]    100
Name: nb, dtype: int64

In [27]:
# Plotting nb of not null per column
@interact_manual
def plot_more_than_n_notna(n =(1,1000,50)):
   ax = nbnotnulldf['nb'][nbnotnulldf['nb']>n].reset_index()\
                .plot.bar(x='index', y='nb', figsize=(25,4))
   ax.set(yscale='log')
   return ax

interactive(children=(IntSlider(value=451, description='n', max=1000, min=1, step=50), Button(description='Run…

In [28]:
# Displaying names of columns with less than 300 entries
mask = pd.cut(nbnotnulldf['nb'], [-1,300])
li_col_del = list(mask[~mask.values.isna()].index)
len(li_col_del), li_col_del

(59,
 ['ingredients_from_palm_oil',
  'cities',
  'ingredients_that_may_be_from_palm_oil',
  '-lignoceric-acid_100g',
  '-elaidic-acid_100g',
  '-gondoic-acid_100g',
  '-mead-acid_100g',
  '-erucic-acid_100g',
  '-nervonic-acid_100g',
  'allergens_en',
  '-melissic-acid_100g',
  'no_nutriments',
  '-caproic-acid_100g',
  'water-hardness_100g',
  '-butyric-acid_100g',
  '-dihomo-gamma-linolenic-acid_100g',
  '-caprylic-acid_100g',
  '-myristic-acid_100g',
  '-stearic-acid_100g',
  'chlorophyl_100g',
  '-montanic-acid_100g',
  'glycemic-index_100g',
  '-cerotic-acid_100g',
  '-palmitic-acid_100g',
  '-capric-acid_100g',
  '-gamma-linolenic-acid_100g',
  '-behenic-acid_100g',
  '-lauric-acid_100g',
  '-maltose_100g',
  'nucleotides_100g',
  'carnitine_100g',
  '-maltodextrins_100g',
  '-arachidic-acid_100g',
  '-oleic-acid_100g',
  'beta-glucan_100g',
  '-arachidonic-acid_100g',
  'additives',
  'serum-proteins_100g',
  'choline_100g',
  'inositol_100g',
  'casein_100g',
  'nutrition-scor

In [29]:
# Printing remaining columns
for s in [s for s in list(df.columns) if s not in li_col_del]:
  print(s, end='|')

code|url|creator|created_t|created_datetime|last_modified_t|last_modified_datetime|product_name|generic_name|quantity|packaging|packaging_tags|brands|brands_tags|categories|categories_tags|categories_en|origins|origins_tags|manufacturing_places|manufacturing_places_tags|labels|labels_tags|labels_en|emb_codes|emb_codes_tags|first_packaging_code_geo|cities_tags|purchase_places|stores|countries|countries_tags|countries_en|ingredients_text|allergens|traces|traces_tags|traces_en|serving_size|serving_quantity|additives_n|additives_tags|additives_en|ingredients_from_palm_oil_n|ingredients_from_palm_oil_tags|ingredients_that_may_be_from_palm_oil_n|ingredients_that_may_be_from_palm_oil_tags|nutriscore_score|nutriscore_grade|nova_group|pnns_groups_1|pnns_groups_2|brand_owner|main_category|main_category_en|energy-kj_100g|energy-kcal_100g|energy_100g|energy-from-fat_100g|fat_100g|saturated-fat_100g|monounsaturated-fat_100g|polyunsaturated-fat_100g|omega-3-fat_100g|-alpha-linolenic-acid_100g|omega-

We keep columns with more than 300 values except:
- 'carbon-footprint_100g', 'carbon-footprint-from-meat-or-fish_100g',
'fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-estimate_100g', as they would require more data to be exploitable

We drop columns with less than 300 values except:
- 'ph_100g' as it could be useful to study a subset of mineral waters

In [30]:
# Putting back pH as it could be useful to study mineral waters for example
li_col_del.remove('ph_100g')
# Adding 4 columns from the "to-be-deleted" columns list 
li_col_del += ['carbon-footprint_100g',
               'carbon-footprint-from-meat-or-fish_100g',
               'fruits-vegetables-nuts_100g',
               'fruits-vegetables-nuts-estimate_100g']
print(f'nb of columns to drop: {len(li_col_del)}')

nb of columns to drop: 62


In [31]:
print("shape before", df.shape)
df.drop(columns=li_col_del, inplace=True)
print("shape after", df.shape)

shape before (1219248, 172)
shape after (1219248, 110)


We rearrange the columns with similar contents at the same place:

In [0]:
# Function that moves similar categories at the same place
def move_cat_containing(my_index, strings):
  idx_sel = []
  for s in strings:
    idx_sel += [i for i,x in enumerate(my_index)\
             if x in my_index[my_index.str.contains(s)]]
  to_move = my_index[idx_sel]
  rank = max(idx_sel)
  mod_index = my_index.drop(to_move)
  for c in to_move:
    mod_index = mod_index.insert(rank,c)
  return mod_index

In [33]:
# Moving 'nutriscores' and 'categories'
cols = df.columns
cols = move_cat_containing(cols,['categ','pnns'])
cols = move_cat_containing(cols,['nutri', 'nova'])
list(cols)

['code',
 'url',
 'creator',
 'created_t',
 'created_datetime',
 'last_modified_t',
 'last_modified_datetime',
 'product_name',
 'generic_name',
 'quantity',
 'packaging',
 'packaging_tags',
 'brands',
 'brands_tags',
 'origins',
 'origins_tags',
 'manufacturing_places',
 'manufacturing_places_tags',
 'labels',
 'labels_tags',
 'labels_en',
 'emb_codes',
 'emb_codes_tags',
 'first_packaging_code_geo',
 'cities_tags',
 'purchase_places',
 'stores',
 'countries',
 'countries_tags',
 'countries_en',
 'ingredients_text',
 'allergens',
 'traces',
 'traces_tags',
 'traces_en',
 'serving_size',
 'serving_quantity',
 'additives_n',
 'additives_tags',
 'additives_en',
 'ingredients_from_palm_oil_n',
 'ingredients_from_palm_oil_tags',
 'ingredients_that_may_be_from_palm_oil_n',
 'ingredients_that_may_be_from_palm_oil_tags',
 'brand_owner',
 'energy-kj_100g',
 'energy-kcal_100g',
 'energy_100g',
 'energy-from-fat_100g',
 'fat_100g',
 'saturated-fat_100g',
 'pnns_groups_2',
 'pnns_groups_1',
 'mai

In [34]:
df = df.reindex(columns=cols)
df.head(1)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,traces,traces_tags,traces_en,serving_size,serving_quantity,additives_n,additives_tags,additives_en,...,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,selenium_100g,iodine_100g,ph_100g,cocoa_100g,phylloquinone_100g,nutriscore_score,nutriscore_grade,nova_group,nutrition-score-fr_100g
0,0000000000017,http://world-en....,kiliweb,8,2018-06-15T10:38...,-90,2019-06-25T11:55...,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,...,nan,nan,nan,8,1,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan


### 1.3 Columns with redundant information

Some of the columns seem to contain similar information. For example : 
  - 'created_t' vs.	'created_datetime'
  - 'last_modified_t' vs.	'last_modified_datetime'
  - 'brands' vs.	'brands_tags'
  - 'categories	' vs.	'categories_tags'  vs.	'categories_en'  
  - 'origins' vs.	'origins_tags'
  - 'manufacturing_places' vs.	'manufacturing_places_tags'
  - 'labels' vs.	'labels_tags'  vs.	'labels_en' 
  - 'emb_codes' vs.	'emb_codes_tags'
  - 'countries' vs.	'countries_tags'  vs.	'countries_en'
  - 'traces' vs.	'traces_tags'  vs.	'traces_en'
  - 'additives_n' vs. 'additives' vs.	'additives_tags'  vs.	'additives_en'


Let's see if some of them can be dropped or their content gathered in only one column.
To that end, we define a function '**comp_df**' that :
- compares the content of two columns col1 and col2 of the dataframe
- prints the number or rows with 1) both columns, 2) col1 only, 3) col2 only, and 4) none of them filled
- returns indexes of 1) data in both columns, 2) data only in col1, 3) data only in col2, and 4) no data at all.

After that, we check and compare the content of the columns when both are filled in order to get familiar with the content of the database and determine the columns that can be dropped.

In [0]:
def comp_df (df, col1, col2, print_Y_N):
  mboth = (~df[[col1,col2]].isna()).sum(axis=1)==2 # mask for both cells filled
  mone = (~df[[col1,col2]].isna()).sum(axis=1)==1 # mask for only one cell filled
  mcol1 = mone & ~df[col1].isna() # mask for only col1 cell filled
  mcol2 = mone & ~df[col2].isna() # mask for only col2 cell filled
  mnone = ~mone & ~mboth
  ind_both = df.index[mboth].to_list()
  ind_col1 = df.index[mcol1].to_list()
  ind_col2 = df.index[mcol2].to_list()
  ind_none = df.index[mnone].to_list()
  if print_Y_N :
    print("nb rows both filled: ", len(ind_both))
    print("nb rows with only", col1, "filled: ", len(ind_col1))
    print("nb rows with only", col2, "filled: ", len(ind_col2))
    print("nb rows not filled: ", len(ind_none))
  else:
    pass
  return(ind_both, ind_col1, ind_col2, ind_none)

In [36]:
# Printing amount of rows in each case:
#(col1 filled, col2 filled, both filled, none filled)
li_col = ['packaging', 'brands', 'categories', 'origins', 'manufacturing_places',
        'labels', 'emb_codes','countries', 'traces']
for c in li_col:
  c1 = c
  c2 = c+'_tags'
  print('COLUMN', c,':')
  t_ind = comp_df(df, c1, c2, True)
  df.loc[t_ind[1], [c1,c2]] = np.nan

COLUMN packaging :
nb rows both filled:  212391
nb rows with only packaging filled:  11
nb rows with only packaging_tags filled:  2
nb rows not filled:  1006844
COLUMN brands :
nb rows both filled:  759635
nb rows with only brands filled:  57
nb rows with only brands_tags filled:  8
nb rows not filled:  459548
COLUMN categories :
nb rows both filled:  546038
nb rows with only categories filled:  3
nb rows with only categories_tags filled:  0
nb rows not filled:  673207
COLUMN origins :
nb rows both filled:  58912
nb rows with only origins filled:  102
nb rows with only origins_tags filled:  3
nb rows not filled:  1160231
COLUMN manufacturing_places :
nb rows both filled:  93930
nb rows with only manufacturing_places filled:  37
nb rows with only manufacturing_places_tags filled:  0
nb rows not filled:  1125281
COLUMN labels :
nb rows both filled:  306649
nb rows with only labels filled:  18
nb rows with only labels_tags filled:  39
nb rows not filled:  912542
COLUMN emb_codes :
nb rows

We design an interactive feature that helps to browse between



Here is an example of the code we have used to address 'brands' and 'brands_tags' columns:

In [37]:
@interact
def print_comp_info(col=li_col):
  c1 = col
  c2 = c1+'_tags'
  # Comparing content of 2 columns, print info and returns indexes 
  t_ind = comp_df(df, c1, c2, True)
  # Displaying a random sample of cells:
  '''- when col1 only is filled
  - when col2 only is filled
  - when both columns are filled'''
  display_side_by_side(df.loc[t_ind[1],[c1, c2]]\
                       .sample(5 if len(t_ind[1])>5 else len(t_ind[1])),
                      df.loc[t_ind[2], [c1, c2]]\
                       .sample(5 if len(t_ind[2])>5 else len(t_ind[2])),
                      df.loc[t_ind[0], [c1, c2]]\
                       .sample(5 if len(t_ind[0])>5 else len(t_ind[0])))

interactive(children=(Dropdown(description='col', options=('packaging', 'brands', 'categories', 'origins', 'ma…

After examination of all the columns above-listed, we decide :
- to set values of the 'XXX' column to NaN if not matching with a non NaN value in the 'XXX_tags' column (as we found that they correspond to erroneous values ('?', '/', '-', etc.));
- to drop 'created_t' and 'last_modified_t';
- to drop 'packaging_tags', 'brands_tags', 'categories_tags', 'origins_tags', 'manufacturing_places_tags', 'labels_tags', 'emb_codes_tags', 'countries_tags', 'traces_tags', 'additives_tags', as the data they contain are redundant with 'XXX' or 'XXX_en' columns; 
- to drop 'additives' column, as it does not contain enough data (29 entries), that are however in english in 'additives_en'
- to keep 'XXX_en' columns, as they contain useful and sometimes original data;
- to keep 'XXX' columns.

In [0]:
# Setting values of the 'XXX' column to NaN if no value in 'XXX_tag' 
t_ind = comp_df(df, c1, c2, False)
df.loc[t_ind[1], [c1,c2]] = np.nan

In [39]:
# Dropping unuseful columns
li_col_del = ['created_t', 'last_modified_t','packaging_tags',
             'brands_tags', 'categories_tags', 'origins_tags', 
             'manufacturing_places_tags', 'labels_tags',
             'emb_codes_tags','countries_tags', 'traces_tags',
             'additives_tags']

print("shape before", df.shape)
df.drop(columns=li_col_del, inplace=True)
print("shape after", df.shape)

shape before (1219248, 110)
shape after (1219248, 98)


In [40]:
# Checking the remaining columns
list(df.columns)

['code',
 'url',
 'creator',
 'created_datetime',
 'last_modified_datetime',
 'product_name',
 'generic_name',
 'quantity',
 'packaging',
 'brands',
 'origins',
 'manufacturing_places',
 'labels',
 'labels_en',
 'emb_codes',
 'first_packaging_code_geo',
 'cities_tags',
 'purchase_places',
 'stores',
 'countries',
 'countries_en',
 'ingredients_text',
 'allergens',
 'traces',
 'traces_en',
 'serving_size',
 'serving_quantity',
 'additives_n',
 'additives_en',
 'ingredients_from_palm_oil_n',
 'ingredients_from_palm_oil_tags',
 'ingredients_that_may_be_from_palm_oil_n',
 'ingredients_that_may_be_from_palm_oil_tags',
 'brand_owner',
 'energy-kj_100g',
 'energy-kcal_100g',
 'energy_100g',
 'energy-from-fat_100g',
 'fat_100g',
 'saturated-fat_100g',
 'pnns_groups_2',
 'pnns_groups_1',
 'main_category_en',
 'main_category',
 'categories_en',
 'categories',
 'monounsaturated-fat_100g',
 'polyunsaturated-fat_100g',
 'omega-3-fat_100g',
 '-alpha-linolenic-acid_100g',
 'omega-6-fat_100g',
 'trans

In [0]:
# Changing the name of some of the columns
df.rename(columns={'nutriscore_score': 'nutriscore',
                       'nutriscore_grade': 'nutrigrade',
                       'pnns_groups_1': 'pnns_gp_1',
                       'pnns_groups_2': 'pnns_gp_2',
                       'nutrition-score-fr_100g':'nutriscore-fr'},
              inplace=True)

Some columns are related and provide information useful to retrieve data via feature engineering.
To help automatizing these tasks, we categorize the columns as follows :

**li_cat_fat**: 'saturated_fat_100g', 'monounsaturated_fat_100g', 'polyunsaturated_fat_100g', 'trans-fat_100g', 'cholesterol_100g'

**li_cat_poly_fat**: 'omega-3-fat_100g', 'omega-6-fat_100g'

**li_cat_carb**: 'sugars_100g', 'starch_100g', 'polyols_100g', 'fibers_100g', '-lactose_100g'

**li_cat_comp**: cat_fat, cat_carb, 'proteins_100g'

**li_cat_vit-oligo**:  'vitamin-a_100g',	'vitamin-d_100g',	'vitamin-e_100g',	'vitamin-k_100g',	'vitamin-c_100g',	'vitamin-b1_100g',	'vitamin-b2_100g',	'vitamin-pp_100g',	'vitamin-b6_100g',	'vitamin-b9_100g',	'folates_100g',	'vitamin-b12_100g',	'biotin_100g',	'pantothenic-acid_100g','silica_100g', 'bicarbonate_100g',	'potassium_100g',	'chloride_100g',	'calcium_100g',	'phosphorus_100g',	'iron_100g',	'magnesium_100g',	'zinc_100g',	'copper_100g',	'manganese_100g',	'fluoride_100g', 'selenium_100g, 'chromium_100g', 'molybdenum_100g', 'iodine_100g',	'caffeine_100g', 'taurine_100g', 'cocoa_100g'

In [0]:
# # Making list of columns with similar or complementary content
# li_cat_carb = ['sugars_100g', 'starch_100g', 'polyols_100g',
#             'fiber_100g', '-lactose_100g']
# li_cat_fat = ['saturated-fat_100g', 'monounsaturated-fat_100g',
#               'polyunsaturated-fat_100g', 'trans-fat_100g', 'cholesterol_100g']
# li_cat_poly_fat = ['omega-3-fat_100g', 'omega-6-fat_100g']
# li_cat_comp = ['fat_100g','carbohydrates_100g', 'proteins_100g']
# li_cat_vit_oligo = ['vitamin-e_100g', 'vitamin-k_100g',
#                     'vitamin-c_100g', 'vitamin-b1_100g', 'vitamin-b2_100g',
#                     'vitamin-pp_100g','vitamin-b6_100g', 'vitamin-b9_100g',
#                     'folates_100g', 'vitamin-b12_100g', 'biotin_100g',
#                     'pantothenic-acid_100g', 'bicarbonate_100g',
#                     'potassium_100g', 'chloride_100g', 'calcium_100g',
#                     'phosphorus_100g', 'iron_100g', 'magnesium_100g',
#                     'zinc_100g', 'copper_100g', 'manganese_100g',
#                     'selenium_100g', 'iodine_100g', 'cocoa_100g']
# li_cat_nutri = ['nutrigrade', 'nova_group','nutriscore', 'nutriscore-fr']
# li_cat_keys = ['code', 'url', 'creator',
#                'created_datetime', 'last_modified_datetime', 
#                'product_name', 'generic_name', 'quantity', 'packaging',
#                'brands','serving_size', 'serving_quantity']
# li_cat_cat = ['categories', 'categories_en', 'labels', 'labels_en',
#               'pnns_gp_1', 'pnns_gp_2', 'main_category',
#               'main_category_en']
# li_cat_geo = ['origins', 'manufacturing_places', 'emb_codes',
#               'first_packaging_code_geo', 'cities_tags', 'purchase_places',
#               'stores', 'countries', 'countries_en']
# li_ca_content = ['ingredients_text', 'allergens', 'traces', 'traces_en',
#                  'additives_n', 'additives_en', 'ingredients_from_palm_oil_n',
#                  'ingredients_from_palm_oil_tags',
#                  'ingredients_that_may_be_from_palm_oil_n',
#                  'ingredients_that_may_be_from_palm_oil_tags']

In [0]:
# # Checking if the columns are contained in the df.columns list
# for li in [li_cat_carb, li_cat_fat, li_cat_poly_fat, li_cat_comp,
#            li_cat_vit_oligo, li_cat_nutri, li_cat_keys, li_cat_cat,
#            li_cat_geo, li_ca_content]:
#   _ = np.array([True if c in df.columns else False for c in li])
#   print("----------\n", li, ":\n", np.sum(_), '/' , len(_), '|', _)

In [0]:
# # Interactive graph to check the number of missing values in available groups
# @interact_manual
# def show_msno_matrix(col_groups=[li_cat_carb, li_cat_fat, li_cat_poly_fat,
#                                  li_cat_comp, li_cat_vit_oligo, li_cat_nutri,
#                                  li_cat_keys, li_cat_cat,li_cat_geo,
#                                  li_ca_content],
#                      color=['b','r', 'g', 'y', 'k', 'p', 'c', 'm' ],
#                      plt_type=['msno matrix', 'msno bars', 'msno heatmap']):
#   idx_subset = pd.Index(col_groups)
#   if plt_type=='msno matrix':
#     return msno.matrix(df.loc[:,idx_subset],figsize=(20,5),
#                      color=colors.to_rgb(color))
#   elif plt_type=='msno bars':
#     return msno.bar(df.loc[:,idx_subset],figsize=(20,5),
#                      color=colors.to_rgb(color))
#   else:
#     return msno.heatmap(df.loc[:,idx_subset],figsize=(20,5))

In [45]:
speak('Filtered out unimportant columns')

In [46]:
print(df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1219248 entries, 0 to 1219247
Data columns (total 98 columns):
 #   Column                                      Non-Null Count    Dtype   
---  ------                                      --------------    -----   
 0   code                                        1219248 non-null  object  
 1   url                                         1219248 non-null  object  
 2   creator                                     1219244 non-null  category
 3   created_datetime                            1219248 non-null  object  
 4   last_modified_datetime                      1219248 non-null  object  
 5   product_name                                1159421 non-null  object  
 6   generic_name                                99460 non-null    category
 7   quantity                                    380072 non-null   category
 8   packaging                                   212391 non-null   category
 9   brands                                      75

In [0]:
# Exporting the dataframe as CSV for further treatment
df.to_csv('OFF_up_to_1_3.csv')

In [0]:
# Exporting dtypes for optimization of memory usage with pandas
df_type = (df.dtypes).to_frame()
type_to_str = lambda x: re.findall(r'[a-z0-9]+', str(x))[1]
d_types = df_type.apply(type_to_str, axis=1).to_dict()

In [0]:
with open('d_types.json', 'w') as outfile:
    json.dump(d_types, outfile)

In [0]:
!pip freeze > requirements.txt

In [51]:
speak('all exportations done')